# Wide&Deep

Wide&Deep 第一层是全连接层，把实值型变量和经过 Embedding 之后的类别型变量拼接在一起后输入到第一层，之后的几层都是 RelU 激活函数的全连接层。同时 Embedding 后的向量还会输入到交叉积层，进行两辆交叉，最终把交叉积层的输出和 RelU 的输出拼接到一起，送入最后的全连接输出层。

In [1]:
# build train data

import os
import numpy as np

BASEDIR = os.getcwd()

fields_dict = {}
lines = None

with open(os.path.join(BASEDIR, 'assets/datasets/criteo_ctr/small_train.txt')) as f:
    lines = f.readlines()

fields_dict = {}
for line in lines:
    line = line.strip('\n')

    for elem in line.split(' ')[1:]:
        field, feature, _ = elem.split(':')

        if field not in fields_dict:
            fields_dict[field] = {'index': len(fields_dict), 'features': {}, 'last_idx': -1}

        if feature not in fields_dict[field]['features']:
            fields_dict[field]['features'][feature] = fields_dict[field]['last_idx'] + 1
            fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1

for field in fields_dict.keys():
    if 'none' not in fields_dict[field]['features']:
        fields_dict[field]['features']['none'] = fields_dict[field]['last_idx'] + 1
        fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1


def init_field_tensor(fields_dict):
    init_tensor = np.zeros((len(fields_dict), 1))
    for field in fields_dict.keys():
        init_tensor[fields_dict[field]['index']] = fields_dict[field]['last_idx']
    return init_tensor.astype(int)


X_train = []
y_train = []

for line in lines:
    line = line.strip('\n')
    elems = line.split(' ')
    y_train.append(float(elems[0]))

    init_tensors = init_field_tensor(fields_dict)
    for elem in elems[1:]:
        field, feature, _ = elem.split(':')
        field_idx = fields_dict[field]['index']
        feature_idx = fields_dict[field]['features'][feature]
        init_tensors[field_idx] = feature_idx
    X_train.append(init_tensors)

X_train = np.concatenate(X_train, 1)
y_train = np.array(y_train)

In [2]:
# build Wide&Deep Network

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class FieldEmbeddingBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FieldEmbeddingBlock, self).__init__()
        self.fields_dict = fields_dict
        self.fields_embedding = {}
        self.embedding_size = embedding_size

        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            self.fields_embedding[field_idx] = nn.Embedding(len(self.fields_dict[field]['features']),
                                                            self.embedding_size)

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[0] * self.embedding_size, input_field_tensor.shape[1])
        idx = 0
        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            out[idx:idx + self.embedding_size, :] = self.fields_embedding[field_idx](
                input_field_tensor[field_idx, :])
            idx += self.embedding_size
        return out.double()


class OuterBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(OuterBlock, self).__init__()
        self.field_cnt = len(fields_dict)
        self.fields_dict = fields_dict
        self.embedding_size = embedding_size
        self.output_dimension = self.embedding_size * self.embedding_size * self.field_cnt

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[1], self.output_dimension)
        p = torch.zeros(self.embedding_size, input_field_tensor.shape[1])
        i = 0
        for _ in range(self.field_cnt):
            p = p + input_field_tensor[i:i + self.embedding_size, :]
            i *= self.embedding_size

        p = p.T
        p = torch.unsqueeze(p, 1)
        q = torch.transpose(p, 1, 2)
        p = q.matmul(p)

        idx1 = 0
        idx2 = 0
        step2 = self.embedding_size * self.embedding_size
        for _ in self.fields_dict.keys():
            out[:, idx2:idx2 + step2] = (input_field_tensor[idx1:idx1 + self.embedding_size, :].T.matmul(p)).reshape(
                (input_field_tensor.shape[1], -1))
            idx2 += step2
            idx1 += self.embedding_size
        return out


class WideAndDeepNetwork(nn.Module):

    def __init__(self, fields_dict, embedding_size):
        super(WideAndDeepNetwork, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size

        self.field_embedding = FieldEmbeddingBlock(fields_dict, embedding_size)
        self.outer_block = OuterBlock(fields_dict, embedding_size)
        self.wide_block = nn.Sequential(
            nn.Linear(self.embedding_size * self.field_cnt, 1024).double(),
            nn.Linear(1024, 1024, bias=True).double(),
            nn.ReLU(),
            nn.Linear(1024, 512, bias=True).double(),
            nn.ReLU(),
            nn.Linear(512, 256).double(),
            nn.ReLU(),
        )

        self.out_layer = nn.Linear(256 + self.outer_block.output_dimension, 1, bias=True).double()

    def forward(self, x):
        out = self.field_embedding(x)
        out1 = self.wide_block(out.T)
        out2 = self.outer_block(out)
        out = torch.cat([out1.double(), out2.double()], 1)
        out = F.sigmoid(self.out_layer(out))
        return out


In [3]:
# PyTorch Version

import torch.optim as optim

device = torch.device('cpu')
LEARNING_RATE = 1e-3

EPOCH = 10
PRINT_STEP = EPOCH / 10
N = len(y_train)

HIDDEN_DIMENSION = 8
EMBEDDING_SIZE = 8
wide_and_deep_neural_netowrk = WideAndDeepNetwork(fields_dict, EMBEDDING_SIZE)

BATCH_SIZE = 8
loss_fn = nn.BCELoss()
optimizer = optim.Adam(wide_and_deep_neural_netowrk.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):

    index = np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)
    X_batch = torch.from_numpy(X_train[:, index]).long()
    y_batch = torch.from_numpy(y_train[index]).reshape(-1, BATCH_SIZE)

    y_hat = wide_and_deep_neural_netowrk(X_batch).reshape(-1, BATCH_SIZE)
    loss = loss_fn(y_hat, y_batch)

    loss.backward()
    optimizer.step()

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, loss))

EPOCH: 0, loss: 0.001089
EPOCH: 1, loss: 12.500007
EPOCH: 2, loss: 0.000000
EPOCH: 3, loss: 25.000000
EPOCH: 4, loss: 0.000000
EPOCH: 5, loss: 25.000000
EPOCH: 6, loss: 4.877228
EPOCH: 7, loss: 37.500000
EPOCH: 8, loss: 25.000278
EPOCH: 9, loss: 12.500000
